## Pandas (read data)

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('task_01.csv')
print(df.head)

<bound method NDFrame.head of                                                answer  grad
0   white::  I chose the color white for the dogho...     2
1   dark gray :: I chose dark gray because it woul...     2
2   black :: i chose the black color becasue that ...     2
3   white ::  While using white for the dog house ...     2
4   black :: Black because in the eperiment black ...     2
5   white :: this color might make the interior of...     1
6   black :: A dark color like black will affect t...     1
7   white :: The white color does not absorb as mu...     1
8   dark gray ::  i picked dark gray because it st...     1
9   light gray :: the inside wont be too hot nor t...     1
10          white :: white duns not atat so mush heat     0
11  white :: The color white Absorbs the least amo...     0
12  white :: The color of the doghouse is white be...     0
13  light gray :: Different color affect the dogho...     0
14  light gray :: Light gray because the dog will ...     0>


In [3]:
print(df.shape)

(15, 2)


## Preparing Model Answer and Students Answers

In [4]:
model_answer = 'light gray ::  Painting the doghouse light gray would keep t he doghouse cool in the summer but warmer in the winter. I know this beca use the data tells you that the lighter the color of the glass jar, the c ooler the inside of the jar is and the darker the lid is the higher the t emparature while be in the glass jar'

answers = df['answer']
print(type(answers))

<class 'pandas.core.series.Series'>


## spaCy semantic similarity

In [32]:
import spacy
nlp = spacy.load('en_core_web_lg')

In [6]:
model_answer_doc = nlp(model_answer)

for answer in answers:
    doc = nlp(answer)
    ratio = doc.similarity(model_answer_doc)
    print('%.2f' % (ratio*3))

# not soo bad

2.94
2.86
2.75
2.85
2.80
2.71
2.83
2.82
2.83
2.67
2.04
2.84
2.80
2.66
2.77


## Cosine Similartiy: Euclidean distance approach
### (counting the maximum number of common words between the documents)

In [54]:
import sklearn.metrics.pairwise as pw
from sklearn.metrics.pairwise import cosine_similarity

from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer()

In [8]:
def calc_cosine_similarity(answers, model_answer):
    # put all data in a list
    data_list = list(answers)
    data_list.append(model_answer)
    
    # convert data to matrix
    data_matrix = tfidf_vectorizer.fit_transform(data_list)
    
    # save model answer vector
    model_vector = data_matrix[-1]
    
    # convert matrix to datafarame to see the word frequencies
    data_term_matrix = data_matrix.todense()
    draft_df = pd.DataFrame(data_term_matrix, columns=tfidf_vectorizer.get_feature_names())
    print(draft_df)
    
    # similarity list
    similarity_list = []
    
    # get similarities
    for vector in data_matrix[:-2]:
        similarity = cosine_similarity(vector, model_vector)
        print('%.2f' % similarity[0][0])
        
        similarity_list.append(similarity[0][0]*3)
    
calc_cosine_similarity(answers, model_answer)

          11        42        47        48        52        53        54  \
0   0.070934  0.061774  0.000000  0.000000  0.000000  0.061774  0.000000   
1   0.000000  0.000000  0.266403  0.133201  0.000000  0.000000  0.000000   
2   0.000000  0.000000  0.000000  0.000000  0.130525  0.227341  0.130525   
3   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
4   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
5   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
6   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
7   0.000000  0.212825  0.000000  0.000000  0.000000  0.000000  0.000000   
8   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
9   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
10  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
11  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
12  0.000000

## Soft Cosine Similarity
### consider the semantic meaning (FastText embedding)

In [12]:
import gensim
from gensim.matutils import softcossim 
from gensim import corpora
import gensim.downloader as api
from gensim.utils import simple_preprocess
print(gensim.__version__)

# Download the FastText model
# fasttext_model300 = api.load('fasttext-wiki-news-subwords-300')

# from gensim.models.wrappers import FastText
# model = FastText.load_fasttext_format('wiki-news-300d-1M-subword')

3.8.0


In [ ]:
documents = list(answers)
documents.append(model_answer)

# Prepare a dictionary and a corpus.
dictionary = corpora.Dictionary([simple_preprocess(doc) for doc in documents])

# Prepare the similarity matrix
# similarity_matrix = fasttext_model300.similarity_matrix(dictionary, tfidf=None, threshold=0.0, exponent=2.0, nonzero_limit=10

## Fuzzy Wuzzy

In [18]:
from fuzzywuzzy import fuzz

In [24]:
for answer in answers:
    ratio = fuzz.token_set_ratio(answer, model_answer)
    print(ratio)
    
# seems good

64
56
58
58
47
46
54
54
43
69
23
51
48
68
55


In [20]:
from strsimpy.levenshtein import Levenshtein
levenshtein = Levenshtein()

for answer in answers:
    print(levenshtein.distance(answer, model_answer))
    
# can't understand

465
213
227
196
214
269
214
239
232
272
284
224
240
249
254


In [23]:
from strsimpy.normalized_levenshtein import NormalizedLevenshtein
normalized_levenshtein = NormalizedLevenshtein()

for answer in answers:
    print('normalized_levenshtein_distance: ', normalized_levenshtein.distance(answer, model_answer))
    
for answer in answers:
    print('normalized_levenshtein_similarity: ', normalized_levenshtein.similarity(answer, model_answer))
    
# Very bad

normalized_levenshtein_distance:  0.6899109792284867
normalized_levenshtein_distance:  0.6677115987460815
normalized_levenshtein_distance:  0.7115987460815048
normalized_levenshtein_distance:  0.6144200626959248
normalized_levenshtein_distance:  0.670846394984326
normalized_levenshtein_distance:  0.8432601880877743
normalized_levenshtein_distance:  0.670846394984326
normalized_levenshtein_distance:  0.7492163009404389
normalized_levenshtein_distance:  0.7272727272727273
normalized_levenshtein_distance:  0.8526645768025078
normalized_levenshtein_distance:  0.890282131661442
normalized_levenshtein_distance:  0.7021943573667712
normalized_levenshtein_distance:  0.7523510971786834
normalized_levenshtein_distance:  0.780564263322884
normalized_levenshtein_distance:  0.7962382445141066
normalized_levenshtein_similarity:  0.31008902077151335
normalized_levenshtein_similarity:  0.33228840125391845
normalized_levenshtein_similarity:  0.28840125391849525
normalized_levenshtein_similarity:  0.385

In [25]:
from strsimpy.jaro_winkler import JaroWinkler
jarowinkler = JaroWinkler()

for answer in answers:
    print(jarowinkler.similarity(answer, model_answer))
    
# not soo bad

0.6667781398475731
0.6820745771895198
0.6996770224497025
0.7276719522338517
0.6516570964753798
0.5807410979824773
0.6780682837175868
0.6325241358800535
0.6248073703579822
0.6339977524102443
0.579427581109667
0.6698087957616088
0.6064938058311211
0.5995425236384025
0.6090463735821027


In [26]:
from strsimpy.longest_common_subsequence import LongestCommonSubsequence
lcs = LongestCommonSubsequence()

for answer in answers:
    print(lcs.distance(answer, model_answer))

557
299
317
269
239
280
275
293
267
278
290
284
260
267
270


In [28]:
from strsimpy.metric_lcs import MetricLCS
metric_lcs = MetricLCS()

for answer in answers:
    print(metric_lcs.distance(answer, model_answer))
    
# soo bad

0.6765578635014837
0.554858934169279
0.5579937304075235
0.5391849529780564
0.6489028213166144
0.8369905956112853
0.6363636363636364
0.7304075235109717
0.7241379310344828
0.8526645768025078
0.890282131661442
0.677115987460815
0.7398119122257053
0.780564263322884
0.7931034482758621


In [30]:
from strsimpy.ngram import NGram
gram = NGram(4)

for answer in answers:
    print(gram.distance(answer, model_answer))

0.7298466864490603
0.7163009404388715
0.7332810867293625
0.6739811912225705
0.6956635318704284
0.7068965517241379
0.7168234064785789
0.7476489028213166
0.6943573667711599
0.6896551724137931
0.7594043887147336
0.7609717868338558
0.7170846394984326
0.664576802507837
0.7037617554858934


In [31]:
from strsimpy.qgram import QGram
qgram = QGram(2)

for answer in answers:
    print(qgram.distance(answer, model_answer))

435
292
266
237
250
271
246
274
277
279
305
261
263
266
275


# 2._Read Mohler data_

In [3]:
instructor_data = pd.read_csv('instructor_data.csv')
print(instructor_data.head)

<bound method NDFrame.head of                                             questions  \
0    What is the role of a prototype program in pr...   
1    What stages in the software life cycle are in...   
2    What are the main advantages associated with ...   
3     Where do C plus plus programs begin to execute?   
4                                 What is a variable?   
..                                                ...   
82                                   What is a queue?   
83   What are the main operations associated with ...   
84        What is the Euler tour traversal of a tree?   
85   How do you delete a node from a binary search...   
86   How many steps does it take to search a node ...   

                                              answers  
0    To simulate the behaviour of portions of the ...  
1    The testing stage can influence both the codi...  
2                        Abstraction and reusability.  
3                               At the main function.  
4    

In [7]:
model_answers = instructor_data.answers
print(model_answers)

0      To simulate the behaviour of portions of the ...
1      The testing stage can influence both the codi...
2                          Abstraction and reusability.
3                                 At the main function.
4          A location in memory that can store a value.
                            ...                        
82     A data structure that stores elements followi...
83                                         push and pop
84     A walk around the tree, starting with the roo...
85     Find the node, then replace it with the leftm...
86                              The height of the tree.
Name: answers, Length: 87, dtype: object


## _read student answers_

In [12]:
import os

In [29]:
answers_dir = 'student_answers/'

## _first solution: os.walk_

In [47]:
answers_list1 = []

for subdir, dirs, files in os.walk(answers_dir):
    # print(subdir)
    # print(dirs)
    # print(files)
    
    for file in files:
        student_answers = pd.read_csv('student_answers/' + file)
        answers_list1.append(student_answers)

print(answers_list1[0])

                                              answers  score
0    High risk problems are address in the prototy...    3.5
1    To simulate portions of the desired final pro...    5.0
2    A prototype program simulates the behaviors o...    4.0
3    Defined in the Specification phase a prototyp...    5.0
4    It is used to let the users have a first idea...    3.0
5    To find problem and errors in a program befor...    2.0
6    To address major issues in the creation of th...    2.5
7    you can break the whole program into prototyp...    5.0
8    -To provide an example or model of how the fi...    3.5
9    Simulating the behavior of only a portion of ...    5.0
10   A program that stimulates the behavior of por...    5.0
11   A program that simulates the behavior of port...    5.0
12   To lay out the basics and give you a starting...    2.0
13   To simulate problem solving for parts of the ...    4.5
14   A prototype program provides a basic groundwo...    2.0
15   A prototype program

## _second solution: os.listdir_

In [48]:
answers_list2 = []

for file in os.listdir(answers_dir):
    if file.endswith(".csv"):
        # print(file)
        
        student_answers = pd.read_csv('student_answers/' + file)
        # sort values
        student_answers.sort_values(by=['score'], ascending=False, inplace=True)
        student_answers.reset_index(drop=True, inplace=True)
        # add to list
        answers_list2.append(student_answers)

print(answers_list2[0])

                                              answers  score
0    Simulating the behavior of only a portion of ...    5.0
1    it simulates the behavior of portions of the ...    5.0
2    Program that simulates the behavior of portio...    5.0
3    To simulate portions of the desired final pro...    5.0
4    it simulates the behavior of portions of the ...    5.0
5    A program that simulates the behavior of port...    5.0
6    A program that stimulates the behavior of por...    5.0
7    It simulates the behavior of portions of the ...    5.0
8    you can break the whole program into prototyp...    5.0
9    Defined in the Specification phase a prototyp...    5.0
10   To simulate problem solving for parts of the ...    4.5
11   A prototype program is a part of the Specific...    4.5
12   A prototype program simulates the behaviors o...    4.0
13   -To provide an example or model of how the fi...    3.5
14   High risk problems are address in the prototy...    3.5
15   the prototype progr

## _spaCy similarity on Mohler data_

In [53]:
model_answer_doc = nlp(model_answers[0])
answers = answers_list2[0]['answers']

for answer in answers:
    doc = nlp(answer)
    ratio = doc.similarity(model_answer_doc)
    
    print('%.2f' % (ratio*5))
    
# soo bad

4.88
4.89
4.83
4.43
4.89
4.84
4.80
4.92
4.41
4.64
4.29
4.45
4.81
4.41
4.35
4.40
4.25
4.53
4.47
4.35
4.30
4.36
4.45
4.47
4.25
4.42
4.09
4.25
4.29


## _Cosine Similartiy on Mohler data_

In [85]:
import math

In [114]:
def calc_cosine_similarity2(answers, model_answer):
    # put all data in a list
    data_list = list(answers)
    data_list.append(model_answer)
    
    # convert data to matrix
    data_matrix = count_vectorizer.fit_transform(data_list)
    
    # save model answer vector
    model_vector = data_matrix[-1]
    
    # convert matrix to datafarame to see the word frequencies
    data_term_matrix = data_matrix.todense()
    draft_df = pd.DataFrame(data_term_matrix, columns=count_vectorizer.get_feature_names())
    # print(draft_df)
    
    # similarity list
    similarity_list = []
    
    # get similarities
    for vector in data_matrix[:-1]:
        similarity = cosine_similarity(vector, model_vector)
        # print( '%.2f' % (similarity[0][0] * 5 + 1) )
        
        similarity_list.append( '%.1f' % ( math.ceil( similarity[0][0] * 5 + 1 ) ) )
    
    return similarity_list



# prepare variables
model_answer = model_answers[0]
answers = answers_list2[0]['answers']
# for test
# print(answers_list2[0]['score'])

# call function
cosine_sim = calc_cosine_similarity2(answers, model_answer)


for index in range(len(cosine_sim)):
    print('question_index: ', index, '\t', 'real score: ', answers_list2[0]['score'][index], '\t', 'estimated score: ', cosine_sim[index])

question_index:  0 	 real score:  5.0 	 estimated score:  5.0
question_index:  1 	 real score:  5.0 	 estimated score:  5.0
question_index:  2 	 real score:  5.0 	 estimated score:  5.0
question_index:  3 	 real score:  5.0 	 estimated score:  4.0
question_index:  4 	 real score:  5.0 	 estimated score:  5.0
question_index:  5 	 real score:  5.0 	 estimated score:  5.0
question_index:  6 	 real score:  5.0 	 estimated score:  5.0
question_index:  7 	 real score:  5.0 	 estimated score:  5.0
question_index:  8 	 real score:  5.0 	 estimated score:  4.0
question_index:  9 	 real score:  5.0 	 estimated score:  5.0
question_index:  10 	 real score:  4.5 	 estimated score:  4.0
question_index:  11 	 real score:  4.5 	 estimated score:  4.0
question_index:  12 	 real score:  4.0 	 estimated score:  5.0
question_index:  13 	 real score:  3.5 	 estimated score:  4.0
question_index:  14 	 real score:  3.5 	 estimated score:  3.0
question_index:  15 	 real score:  3.0 	 estimated score:  4.0
qu

# Accuracy

In [116]:
def calc_accuracy(real_scores, estimated_scores):
    counter = 0
    
    for i in range(len(real_scores)):
        re_sc = real_scores[i]
        es_sc = float(estimated_scores[i])
        if re_sc == es_sc or re_sc-es_sc == 0.5 or es_sc-re_sc == 0.5:
            counter += 1
    
    accuracy = (counter / len(real_scores)) * 100
    return accuracy

accuracy = calc_accuracy(answers_list2[0]['score'], cosine_sim)
print('%.2f' % accuracy, '%')

62.07 %
